In [1]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
path = r'T:\DCProjects\StoryMap\BikeCounting\BikeMap\data\InteractiveBikeMap.gdb'
bike_friendly_businesses = gpd.read_file(path, layer="Bike_Friendly_Businesses")

In [4]:
bike_friendly_businesses.head()

,Status,Score,Match_type,Match_addr,Side,Ref_ID,User_fld,Addr_type,ARC_Street,ARC_City,ARC_State,ARC_ZIP,Account_Na,Street,City,Zip_Postal,Date_Appro,Applicant,Region,geometry
0,M,89.04,M,"91049 S WILLAMETTE ST, Coburg, OR, 97408",,98634,,PointAddress,91049 SOUTH WILLAMETTE; BOX 8542,COBURG,,,Reflections Salon,91049 South Willamette; Box 8542,Coburg,97408,2015-11-25T00:00:00,Vicki Bell,Willamette Valley,POINT (4247692.024 910514.051)
1,M,100.00,A,"91003 S WILLAMETTE ST, Coburg, OR, 97408",,101760,,PointAddress,91003 S Willamette St,Coburg,,,The Oregon Handwork Studio,91003 S Willamette St,Coburg,97408,2016-04-28T00:00:00,Kathy Paterson,Willamette Valley,POINT (4247890.889 910107.994)
2,M,100.00,A,"90999 S WILLAMETTE ST, Coburg, OR, 97408",,134476,,PointAddress,PO Box 8008 90999 S Willamette St,Coburg,,,Coburg Pizza Company,PO Box 8008 90999 S Willamette St,Coburg,97408,2014-11-20T00:00:00,Terry Dawson,Willamette Valley,POINT (4247953.708 909940.685)
3,M,100.00,M,"91136 N WILLAMETTE ST, Coburg, OR, 97408",,134510,,PointAddress,91136 N. WILLAMETTE STREET,COBURG,,,City of Coburg,PO Box 8316,Coburg,97408,2014-10-24T00:00:00,Petra Schuetz,Willamette Valley,POINT (4247265.300 911467.629)
4,U,0.00,A,,,-1,,,500 West 15th,Eugene,,,Fort Dalles Museum,500 West 15th,Eugene,97058,2014-05-12T00:00:00,Trish Neal,Mt. Hood/Gorge,POINT (6052081273769120.000 6052081291735920.000)


In [5]:
bike_friendly_businesses.columns

Index(['Status', 'Score', 'Match_type', 'Match_addr', 'Side', 'Ref_ID',
       'User_fld', 'Addr_type', 'ARC_Street', 'ARC_City', 'ARC_State',
       'ARC_ZIP', 'Account_Na', 'Street', 'City', 'Zip_Postal', 'Date_Appro',
       'Applicant', 'Region', 'geometry'],
      dtype='object')

In [6]:
import ReviewData as RD

In [7]:
site_address = RD.readSiteAddress()

In [8]:
site_address.head()

,address,longitude,latitude,geom
0,18008 SIUSLAW RD,-123.615974,43.919974,POINT (-13760867.236 5453066.379)
1,83908 HWY 101,-124.111965,43.912707,POINT (-13816080.721 5451943.265)
2,83520 HWY 101,-124.116499,43.901309,POINT (-13816585.429 5450182.228)
3,83394 HWY 101,-124.117368,43.898493,POINT (-13816682.165 5449747.154)
4,83380 HWY 101,-124.117573,43.898105,POINT (-13816705.064 5449687.166)


In [9]:
site_address.columns

Index(['address', 'longitude', 'latitude', 'geom'], dtype='object')

In [10]:
business = gpd.read_file(r"X:\Projects\RLID\BusinessData\InfoUSA\2021\October\Geocoded.gdb", layer='Oct2021_Geo')

In [11]:
business.columns

Index(['biz_business_name', 'biz_business_desc', 'biz_address', 'biz_city',
       'biz_state', 'biz_zip', 'biz_zip_plus4', 'biz_zip_full', 'biz_county',
       'biz_carrier_route', 'biz_delivery_point_bar_code', 'biz_neighborhood',
       'biz_phone', 'biz_phone_toll_free', 'biz_fax', 'biz_website',
       'biz_primary_sic_code', 'biz_primary_sic_desc', 'biz_sic_code_1',
       'biz_sic_code_1_desc', 'biz_sic_code_2', 'biz_sic_code_2_desc',
       'biz_sic_code_3', 'biz_sic_code_3_desc', 'biz_sic_code_4',
       'biz_sic_code_4_desc', 'biz_naics_industry_code_group',
       'biz_naics_industry_desc', 'biz_primary_naics',
       'biz_primary_naics_desc', 'biz_naics_1', 'biz_naics_1_desc',
       'biz_naics_2', 'biz_naics_2_desc', 'biz_naics_3', 'biz_naics_3_desc',
       'biz_naics_4', 'biz_naics_4_desc', 'biz_cuisine_code',
       'biz_cuisine_code_desc', 'biz_location_employee_size_code',
       'biz_location_employee_size_range', 'biz_location_employee_size_actual',
       'biz_busi

In [12]:
business.biz_address.head()

0          3879 Cross St
1    900 Mckinley St # D
2     220 E 11th Ave # 5
3       3340 Highway 101
4          50 E 11th Ave
Name: biz_address, dtype: object

In [13]:
business.biz_business_name.head()

0          #1 Northwest Inc
1      1-800-Radiator & A/C
2    1000 Friends Of Oregon
3          101 Mini-Storage
4          1050 Lincoln LLC
Name: biz_business_name, dtype: object

In [20]:
import pandas as pd

In [15]:
bfb = pd.read_excel(r"T:\DCProjects\StoryMap\BikeCounting\BikeMap\21.11.22 BFB_Lane County.xlsx")

In [16]:
bfb.head()

,Business Name,Address,City,State,Zip Code
0,Eugene Vineyard Church,3455 Royal Ave.,Eugene,OR,97402
1,Blair Family & Sports Chiropractic,1185 Arther St.,Eugene,OR,97402
2,Everyday People Yoga,352 W. 12th Avenue,Eugene,OR,97401
3,Jenny's Hair and Company,2833 Willamette #D,Eugene,OR,97405
4,Reflections Salon,91049 South Willamette,Coburg,OR,97408


In [17]:
bfb.shape

(96, 5)

In [18]:
df=bfb[bfb['Business Name'].isin(business.biz_business_name)]

In [34]:
df['Business Name'].values

array(['Everyday People Yoga', 'Reflections Salon', 'Sand Master Park',
       'Cascade Motel', 'Valley River Inn', 'Wayfarer Resort',
       'Sea Lion Caves', 'Arriving By Bike', 'Bike Friday',
       'Co-Motion Cycles', 'Klink Cycles', 'Life Cycle Bike Shop',
       'Simply Cycle', 'Capella Market', 'Main Street Market',
       'Mohawk General Store', 'Sundance Natural Foods', 'Crumb Together',
       '16 Tons Cafe', 'Washburne Cafe', 'Creswell Bakery',
       'Morning Glory Cafe', 'Noisette Pastry Kitchen', 'Pig & Turnip',
       'Viva! Vegetarian Grill', 'Middlefield Golf Course'], dtype=object)

In [62]:
for bizname in df['Business Name'].values:
    df.loc[df['Business Name']==bizname, 'Longitude'] = business.loc[business['biz_business_name']==bizname, 'biz_longitude'].values[0]
    df.loc[df['Business Name']==bizname, 'Latitude'] = business.loc[business['biz_business_name']==bizname, 'biz_latitude'].values[0]

C:\Users\clid1852\.conda\envs\geoenv\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [63]:
df

,Business Name,Address,City,State,Zip Code,Longitude,Latitude
2,Everyday People Yoga,352 W. 12th Avenue,Eugene,OR,97401,-123.098084,44.046307
4,Reflections Salon,91049 South Willamette,Coburg,OR,97408,-123.065833,44.136608
14,Sand Master Park,4981 Oregon Coast Hwy,Florence,OR,97439,-124.102390,44.009984
27,Cascade Motel,47487 Hwy. 58,Oakridge,OR,97463,-122.489153,43.747399
34,Valley River Inn,1000 Valley River Way,Eugene,OR,97401,-123.102253,44.065724
35,Wayfarer Resort,46725 Goodpasture Road,Vida,OR,97488,-122.528835,44.126453
36,Sea Lion Caves,91560 Hwy 101,Florence,OR,97439,-124.125155,44.118995
40,Arriving By Bike,2705 Willamette St,Eugene,OR,97405,-123.090342,44.028776
42,Bike Friday,3364 W 11th Ave.,Eugene,OR,97402,-123.142143,44.047248
43,Co-Motion Cycles,4765 Pacific Ave,Eugene,OR,97402,-123.181784,44.057681


In [19]:
df.shape

(26, 5)

In [20]:
df1=bfb[~bfb['Business Name'].isin(business.biz_business_name)] 

In [21]:
df1.head()

,Business Name,Address,City,State,Zip Code
0,Eugene Vineyard Church,3455 Royal Ave.,Eugene,OR,97402
1,Blair Family & Sports Chiropractic,1185 Arther St.,Eugene,OR,97402
3,Jenny's Hair and Company,2833 Willamette #D,Eugene,OR,97405
5,Lauren's Place,10987 E. Mapleton Road,Mapleton,OR,97453
6,Lily of the Field B&B,35722 Ross Lane,Cottage Grove,OR,97424


In [22]:
df2=df1[df1['Address'].isin(business.biz_address)] 

In [23]:
list1 = df1['Business Name'].tolist()
list2 = business['biz_business_name'].tolist()

In [24]:
mat1 = []
mat2 = []
p = []

In [25]:
threshold = 87

In [26]:
for i in list1:
    mat1.append(process.extract(i, list2, limit=2))
df1['matches'] = mat1

<ipython-input-26-37444a1650d2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['matches'] = mat1


In [27]:
for j in df1['matches']:
    for k in j:
        if k[1] >= threshold:
            p.append(k[0])
    mat2.append(",".join(p))
    p = []

In [28]:
df1.head()

,Business Name,Address,City,State,Zip Code,matches
0,Eugene Vineyard Church,3455 Royal Ave.,Eugene,OR,97402,"[(AC Eugene, 86), (BMW Of Eugene, 86)]"
1,Blair Family & Sports Chiropractic,1185 Arther St.,Eugene,OR,97402,"[(Blair Family & Sports Chiro, 89), (A & J Spo..."
3,Jenny's Hair and Company,2833 Willamette #D,Eugene,OR,97405,"[(A Hair Alchemist, 86), (A R S Fresno, 86)]"
5,Lauren's Place,10987 E. Mapleton Road,Mapleton,OR,97453,"[(A1 Ron s Stump & Tree Removal, 86), (A1 Ron ..."
6,Lily of the Field B&B,35722 Ross Lane,Cottage Grove,OR,97424,"[(Above The Rest, 86), (Access The Law, 86)]"


In [45]:
df1['matches'] = mat2

<ipython-input-45-8594f76d48c2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['matches'] = mat2


In [47]:
df1.to_csv("matches.csv", index=False)

In [48]:
# examine the table matches.csv and review the matches; the table is updated with the edits on the business names
df3 = pd.read_csv("matches1.csv")

In [49]:
df3.shape

(38, 6)

In [68]:
d = []
for bizname in df3['Business Name'].values:
    match = df3.loc[df3['Business Name']==bizname, 'matches'].values[0]
    df3.loc[df3['Business Name']==bizname, 'Longitude'] = business.loc[business['biz_business_name']==match, 'biz_longitude'].values[0]
    df3.loc[df3['Business Name']==bizname, 'Latitude'] = business.loc[business['biz_business_name']==match, 'biz_latitude'].values[0]
    test_df = business[business['biz_business_name']==match]
    if test_df.shape[0] > 1:
        d.append(match)

In [69]:
d

['Paul s Bicycle Way Of Life',
 'Paul s Bicycle Way Of Life',
 'Rolf Prima Inc',
 'Full City Coffee Roasters',
 'Mezza Luna Pizzeria',
 'Mezza Luna Pizzeria',
 'Oakshire Brewing']

In [72]:
ndf = df.append(df3.drop(columns=['matches']))

In [73]:
ndf.to_csv("matched.csv", index=False)

In [74]:
# review the table on the duplicated names
df3[df3.matches.isin(d)]

,Business Name,Address,City,State,Zip Code,matches,Longitude,Latitude
16,Paul's Bicycle Way of Life – Downtown,556 Charnelton Street,Eugene,OR,97401,Paul s Bicycle Way Of Life,-123.152168,44.047777
17,Paul's Bicycle Way of Life – South Eugene,2480 Alder St,Eugene,OR,97405,Paul s Bicycle Way Of Life,-123.152168,44.047777
19,Rolf Prima,150 Shelton McMurphey Blvd. Ste 101,Eugene,OR,97401,Rolf Prima Inc,-123.148654,44.050623
23,Full City Coffee Roasters - High St. Cafe,295 E 13th Ave.,Eugene,OR,97401,Full City Coffee Roasters,-123.154571,44.045338
26,Mezza Luna Pizzeria - Crescent Village,2776 Shadow View Drive,Eugene,OR,97408,Mezza Luna Pizzeria,-123.060329,44.092954
27,Mezza Luna Pizzeria - Downtown,933 Pearl Street,Eugene,OR,97401,Mezza Luna Pizzeria,-123.060329,44.092954
33,Oakshire Brewing Public House,207 Madison Street,Eugene,OR,97402,Oakshire Brewing,-123.140746,44.073006


In [1]:
from geopy.geocoders import Nominatim, GoogleV3

In [2]:
geolocator = Nominatim(user_agent="dongmeijane1107@gmail.com")

In [7]:
data = geolocator.geocode("2650 Willamette St., Eugene, OR")
data.raw

{'place_id': 292170964,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 721761539,
 'boundingbox': ['44.029372571429',
  '44.029472571429',
  '-123.09087',
  '-123.09077'],
 'lat': '44.02942257142857',
 'lon': '-123.09082',
 'display_name': '2650, Willamette Street, Woodfield Microhood, College Hill, Eugene, Lane County, Oregon, 97405, United States',
 'class': 'place',
 'type': 'house',
 'importance': 0.5209999999999999}

In [8]:
data.raw.get("lat"), data.raw.get("lon")

('44.02942257142857', '-123.09082')

In [9]:
coords = data.raw.get("lat"), data.raw.get("lon")

In [17]:
float(coords[0])

44.02942257142857

In [18]:
def getCoordinates(address):
    data = geolocator.geocode(address)
    coords = data.raw.get("lat"), data.raw.get("lon")
    return coords

In [74]:
matches0 = pd.read_csv('matches0.csv')

In [75]:
matches0 = matches0[matches0.City.isin(['Eugene', 'Springfield'])]

In [64]:
matches0.head() 

,Business Name,Address,City,State,Zip Code
0,Eugene Vineyard Church,3455 Royal Ave.,Eugene,OR,97402
1,Jenny's Hair and Company,2833 Willamette St.,Eugene,OR,97405
5,Pony House Inn Suites & Cottages,660 C St.,Springfield,OR,97477
7,Velo Bed and Breakfast,86211 Bailey Hill Road,Eugene,OR,97405
9,Bob Keefer Center for Sports and Recreation,250 S 32nd St.,Springfield,OR,97478


In [76]:
matches0['FullAddress'] = matches0['Address'] + ', ' + matches0['City'] + ', ' + matches0['State']

In [79]:
matches0['Longitude'] = matches0.FullAddress.apply(lambda x: float(getCoordinates(x)[1]))
matches0['Latitude'] = matches0.FullAddress.apply(lambda x: float(getCoordinates(x)[0]))